In [66]:
import tensorflow as tf
model = tf.lite.Interpreter(r'models\hunam_model_float32.tflite')
face_detector = cv.FaceDetectorYN.create(r'models\face_detection_yunet_2023mar.onnx', "", (320, 320))
model.allocate_tensors()
import cv2 as cv
import numpy as np

In [67]:
inp_details = model.get_input_details()[0]
out_details = model.get_output_details()[0]

print(f"inp = {inp_details}")
print(f"out = {out_details}")

inp = {'name': 'images', 'index': 0, 'shape': array([  1, 640, 640,   3], dtype=int32), 'shape_signature': array([  1, 640, 640,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
out = {'name': 'Identity', 'index': 1379, 'shape': array([   1,    6, 8400], dtype=int32), 'shape_signature': array([   1,    6, 8400], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}


In [68]:
def preprocess_img(img_path):
    img = cv.imread(img_path)
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    img = cv.resize(img, (640, 640))
    img = img.astype(np.float32) / 255.0
    return np.expand_dims(img, axis = 0)

In [69]:
def run_inference(img_path):
    img = preprocess_img(img_path)
    print(img.shape)
    model.set_tensor(inp_details['index'], img)
    model.invoke()
    out = model.get_tensor(out_details['index'])
    return np.asarray(out)

In [70]:
res = run_inference(r"C:\Users\s9554\Downloads\test2_3_human.jpg")

(1, 640, 640, 3)


In [71]:
out_data = np.squeeze(res)
fnl = []
for i in range(out_data.shape[1]):
    if 0.46 <= out_data[4, i] <= 0.47:
        x, y, w, h = out_data[: 4, i]
        conf = out_data[4, i]
        class_id = int(out_data[5, i])
        fnl.append([x, y, w, h, conf, class_id])

In [72]:
fnl

[[np.float32(0.3391194),
  np.float32(0.5349584),
  np.float32(0.21574287),
  np.float32(0.8599608),
  np.float32(0.46323875),
  0]]

In [73]:
def post_process(output_data, conf_threshold=0.5):
    output_data = np.squeeze(output_data)  # Remove batch dimension
    boxes = []
    for i in range(output_data.shape[1]):
        conf = output_data[4, i]  # Confidence score
        if conf > conf_threshold:
            # Extract bounding box and class
            x, y, w, h = output_data[:4, i]
            class_id = int(output_data[5, i])
            boxes.append([x, y, w, h, conf, class_id])
    return boxes

In [74]:
boxes = post_process(res)

In [75]:
sor_box = sorted(boxes, reverse = True)

In [76]:
print(len(sor_box))

5


In [77]:
box_high = sor_box[0]
box_high

[np.float32(0.6357984),
 np.float32(0.54204184),
 np.float32(0.17034161),
 np.float32(0.7939236),
 np.float32(0.53402007),
 0]

In [98]:
def draw_img(img_path, box, conf_thres = 0.4):
    image = cv.imread(img_path)
    face_img = []
    h, w, _ = image.shape  # Original image dimensions

    # Denormalize and convert boxes to (x1, y1, x2, y2)
    processed_boxes = []
    for b in box:
        x_center, y_center, width, height, conf, class_id = b
        if conf > conf_thres:
            x_center *= w
            y_center *= h
            width *= w
            height *= h

            # Convert to corner coordinates
            x1 = int(x_center - width / 2)
            y1 = int(y_center - height / 2)
            x2 = int(x_center + width / 2)
            y2 = int(y_center + height / 2)

            cv.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            img_face = image[y1: y2 , x1: x2]
            frame_width = img_face.shape[1]
            frame_height = img_face.shape[0]
            # if frame_height > 320:
            #     crop_h = (frame_height - 320) // 2
            #     img_face = img_face[crop_h: crop_h + 320, :]
            # if frame_width > 320:
            #     crop_w = (frame_width - 320) // 2
            #     img_face = img_face[:, crop_w: crop_w + 320]
            pad_h = max(0, 320 - frame_height)
            pad_w = max(0, 320 - frame_width)
            padded_img = np.pad(img_face, (
                (pad_h // 2, pad_h - pad_h // 2),
                (pad_w //2, pad_w - pad_w // 2),
                (0, 0)),
                mode = 'constant',
                constant_values = 0)
            ph = padded_img.shape[0]
            pw = padded_img.shape[1]
            print(padded_img.shape)
            print(pw)
            print(ph)
            face_detector.setInputSize((pw, ph))
            _, face = face_detector.detect(padded_img)
            # face_img.append(padded_img)
            for f in face:
                bbox = f[:4].astype(int)
                x1, y1, width, height = bbox
                x2, y2 = x1 + width, y1 + height
                cv.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv.putText(image, f"Conf: {conf:.2f}", (x1, y1 - 10),
            cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv.imshow("NMS Results", image)
    cv.waitKey(0)
    cv.destroyAllWindows()

    return face_img

In [99]:
faces = draw_img(r"C:\Users\s9554\Downloads\test2_3_human.jpg", sor_box)

(324, 320, 3)
320
324
(320, 320, 3)
320
320
(320, 320, 3)
320
320
(351, 320, 3)
320
351
(347, 320, 3)
320
347


In [61]:
face = np.asarray(faces)
print(face.shape)

(0,)


In [10]:
def iou(b1, b2):
    x1, y1, x2, y2 = b1
    x3, y3, x4, y4 = b2

    x_iter1 = max(x1, x3)
    y_ter1 = max(y1, y3)
    x_iter2 = min(x2, x4)
    y_iter2 = min(y1, y4)

    width_iter = abs(x_iter2 - x_iter1)
    height_iter = abs(y_iter2 - y_ter1)
    area_iter = width_iter * height_iter

    width_box1 = abs(x2 - x1)
    height_box1 = abs(y2 - y1)
    width_box2 = abs(x4 - x3)
    height_box2 = abs(y4 - y3)

    area_box1 = width_box1 * height_box1
    area_box2 = width_box2 * height_box2

    area_union = area_box2 + area_box1 - area_iter
    iou = area_iter / area_union

    return iou

In [11]:
def non_max_supression(boxes, iou_thres = 0.5):
    boxes = sorted(boxes, key = lambda x: x[4], reverse = True)
    selected_box = []

    while boxes:
        chosen_box = boxes.pop(0)
        selected_box.append(chosen_box)

        # Remove boxes with high overlap (IoU threshold)
        boxes = [box for box in boxes if iou(chosen_box[:4], box[:4]) < iou_thres]

    return selected_box

In [12]:
def draw_boxes_with_nms(image_path, boxes, conf_threshold=0.5):
    # Load the image
    image = cv.imread(image_path)
    h, w, _ = image.shape  # Original image dimensions

    # Denormalize and convert boxes to (x1, y1, x2, y2)
    processed_boxes = []
    for box in boxes:
        x_center, y_center, width, height, conf, class_id = box
        if conf > conf_threshold:
            x_center *= w
            y_center *= h
            width *= w
            height *= h

            # Convert to corner coordinates
            x1 = int(x_center - width / 2)
            y1 = int(y_center - height / 2)
            x2 = int(x_center + width / 2)
            y2 = int(y_center + height / 2)
            processed_boxes.append([x1, y1, x2, y2, conf, class_id])

    # Apply Non-Maximum Suppression (NMS)
    nms_boxes = non_max_supression(processed_boxes)

    # Draw final boxes after NMS
    for box in nms_boxes:
        x1, y1, x2, y2, conf, class_id = box
        cv.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv.putText(image, f"Conf: {conf:.2f}", (x1, y1 - 10),
                    cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the image
    cv.imshow("NMS Results", image)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [13]:
draw_boxes_with_nms(r"C:\Users\s9554\Downloads\test2_3_human.jpg", boxes)